In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from pfs.lam.imageAnalysis import neighbor_outlier_filter

from pfs.lam.detAnalysis import *
from pfs.lam.detFocusAnalysis import *
from pfs.lam.fileHandling import *
from pfs.lam.style import *
from pfs.lam.analysisPlot import *
import glob
import lsst.daf.persistence as dafPersist

In [ ]:
from pfs.lam.misc import find_nearest
import seaborn as sns


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime

In [ ]:
datetime.now().strftime("%Y-%m-%dT%Hh%M")

In [ ]:
roi_size = 24
doBck = True
threshold = 0.

In [ ]:
expIds =[177]
rerun = "sm1-march2020-no_FD"
repo = "repo"
cam = "r1"
extra = ''
roi_size = 24
#drpPath = "/home/fmadec/pfs/lam/drp"

In [ ]:
expIds = [24572]
rerun = "ginga"
cam = "b3"
extra = ''
roi_size = 24


# define defaut parameters


In [ ]:
piston_index = "motor1"

com = True  # Center Of Mass
head = 0
tail = 0
criteria = 'EE5'
criterias = [criteria] #['EE5', 'sep_ECE5']
verbose = True
doPrint = False
arm = cam[0]


In [ ]:
head = 0
tail = 0

In [ ]:
cluster = False

In [ ]:
doForestFilter = False
forest_contamin = 0.05   #r1 0.05   #m1 

In [ ]:
doNeighborFilter = True
thres_neighbor = -0.1 # default -0.1

In [ ]:
doSave = False
doSaveCsv = False

# Set folder according place either cluster, laptop

In [ ]:
drpPath = "/data/drp"
repo = "sps"
basePath = '/drp/analysis/sm3/'
imgPath = basePath


In [ ]:
outpath = ''
outpath = "output\\" if outpath is None else outpath



# Read data CSV files for all experimentIds

In [ ]:
files = []
for experimentId in expIds:

    csvPath = imgPath+"Exp"+str(experimentId)+"/"+rerun+"_roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra
    if not os.path.exists(csvPath):
        csvPath = imgPath+"Exp"+str(experimentId)+"/"+rerun+"/"+"roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra

    dataPath = csvPath
    print(dataPath)

    searchFile = f"{csvPath}Imquality_{cam}_Exp{experimentId}*"
    print(searchFile)
    files.extend(glob.glob(searchFile))

if verbose:
    print(*files, sep="\n")

piston_imdata = pd.concat(map(pd.read_csv,files)).reset_index().sort_values(by="motor1")
#piston_imdata["wavelength"] = 300

piston_imdata = piston_imdata.groupby(['wavelength','fiber', "motor1"]).mean().reset_index()

In [ ]:
# hack debut sm2


print(f"Wavelength used: {piston_imdata.wavelength.unique()}")
minPos = piston_imdata[['motor1','motor2','motor3']]
minPos = minPos - minPos.min()
piston_imdata['relPos'] = minPos['motor1']

#piston = piston_imdata[piston_imdata.EE5>threshold]

piston_imdata["fit_flag"] = 0
piston_imdata["EE5_diff_flag"] = True

piston = piston_imdata

#piston = piston_imdata[piston_imdata.EE5>threshold]

In [ ]:
waves = piston.wavelength.unique()
fibers = piston.fiber.unique()
print(fibers, waves)

# filtering the data

In [ ]:

if doNeighborFilter :
    tot = []
    for  group_name, series in piston.groupby(['wavelength','fiber']):
        series = neighbor_outlier_filter(series, "EE5", thres_neighbor, absolute=True)
        tot.append(series)
        piston_filtered = pd.concat(tot)
        

# 2nd Moment calculation from SEP

In [ ]:
if "sep_x2" in piston_filtered.columns:
    piston_filtered["sep_2ndM"]= piston_filtered.apply(lambda x: np.mean([x["sep_x2"],x["sep_y2"]]) , axis=1)
    piston_filtered["sep_radius_x"]= np.sqrt(piston["sep_x2"])
    piston_filtered["sep_radius_y"]= np.sqrt(piston["sep_y2"])
    piston_filtered["sep_radius_xy"]= piston_filtered.apply(lambda x: np.mean([x["sep_radius_x"],x["sep_radius_y"]]) , axis=1)

In [ ]:
# Plot raw data

In [ ]:
grid = sns.FacetGrid(piston_filtered, col="fiber", hue="wavelength",
#                         col_wrap=4, height=3, legend_out=True)
                     col_wrap=3, height=3, legend_out=True)

grid.map(plt.plot, "relPos", criteria, marker="+")
grid.fig.tight_layout(w_pad=1)
if criteria == "EE5":
    grid.set(ylim=(0, 1))
grid.add_legend()
plt.subplots_adjust(top=0.85)
#if title_suffix is None:
#    title = f"{plot_prefix} - Exp{experimentId}"
#else: 
#    title = f"{plot_prefix} - Exp{experimentId}\n{title_suffix}"
#grid.fig.suptitle(title)

# Through focus fit 

In [ ]:
piston_filtered[piston_filtered.fit_flag >=0 & piston_filtered.EE5_diff_flag].motor1.min()

In [ ]:
thfoc_data = getAllBestFocus(piston_filtered.dropna(), criterias=['EE5'], doPlot=True)

In [ ]:
thfoc_data

In [ ]:
plane = getBestPlane(thfoc_data[thfoc_data.criteria == "EE5"], coords=["px", "py", "focus"], doPlot=True)

In [ ]:
motors_pos = findMotorPos(plane, cam=cam, doPrint=True)

In [ ]:
print(csvPath)
print(f"ExpId {experimentId}")
#print(invMat)
#print("xcu_%s motors moveCcd a=%.2f b=%.2f c=%.2f microns abs"%(cam,*motors_pos))

In [ ]:
selected = thfoc_data[thfoc_data.criteria == "EE5"]
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.scatter(selected.fiber, selected.wavelength, selected.focus, c='r')
ax.set_zlim(0,400)

selected = thfoc_data[thfoc_data.criteria == "sep_2ndM"]

ax.scatter(selected.fiber, selected.wavelength, selected.focus, c='b')
ax.set_zlim(0,400)


In [ ]:
fig = plt.subplot()
thfoc_data.set_index(["fiber", "wavelength"]).groupby("criteria").value.plot()

In [ ]:
fig = plt.subplot()
thfoc_data.set_index(["fiber", "wavelength"]).groupby("criteria").width.plot()
thfoc_data.width.mean()